In [1]:
import Pkg
using JuMP
using HiGHS

model = Model(HiGHS.Optimizer)


A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [2]:
shifts    = 1:6
intervals = 1:6

demand = Dict(
    1 =>  4,   # 00:00–04:00
    2 =>  8,   # 04:00–08:00
    3 => 10,   # 08:00–12:00
    4 =>  7,   # 12:00–16:00
    5 => 12,   # 16:00–20:00
    6 =>  4    # 20:00–24:00
)

coverage = Dict(
    1 => [1, 2],
    2 => [2, 3],
    3 => [3, 4],
    4 => [4, 5],
    5 => [5, 6],
    6 => [6, 1] 
)


Dict{Int64, Vector{Int64}} with 6 entries:
  5 => [5, 6]
  4 => [4, 5]
  6 => [6, 1]
  2 => [2, 3]
  3 => [3, 4]
  1 => [1, 2]

In [3]:
@variable(model, x[shifts] >= 0, Int)


1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:6
And data, a 6-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]

In [4]:
@constraint(model, [j in intervals], 
    sum(x[i] for i in shifts if j in coverage[i]) >= demand[j]
)


1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:6
And data, a 6-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 x[1] + x[6] >= 4
 x[1] + x[2] >= 8
 x[2] + x[3] >= 10
 x[3] + x[4] >= 7
 x[4] + x[5] >= 12
 x[5] + x[6] >= 4

In [5]:

@objective(model, Min, sum(x[i] for i in shifts))

optimize!(model)

println("Número mínimo de autobuses: ", objective_value(model))
for i in shifts
    println("Turno $i → ", Int(value(x[i])), " autobuses")
end


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 6 rows; 6 cols; 12 nonzeros; 6 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [4e+00, 1e+01]
Presolving model
6 rows, 6 cols, 12 nonzeros  0s
6 rows, 6 cols, 12 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   6 rows
   6 cols (0 binary, 6 integer, 0 implied int., 0 continuous, 0 domain fixed)
   12 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |      